### ENVIRONMENT

In [2]:
"""
This is a gym environment based for the drone_game
The goal is to reach randomly positoned targets
"""

import os
from math import sin, cos, pi, sqrt
from random import randrange

import numpy as np
import gym
from gym import spaces

import pygame
from pygame.locals import *


class droneEnv(gym.Env):
    def __init__(self, render_every_frame, mouse_target):
        super(droneEnv, self).__init__()

        self.render_every_frame = render_every_frame
        # Makes the target follow the mouse
        self.mouse_target = mouse_target

        # Initialize Pygame, load sprites
        pygame.init()
        self.screen = pygame.display.set_mode((800, 800))
        self.FramePerSec = pygame.time.Clock()

        self.player = pygame.image.load(os.path.join("assets/sprites/drone_old.png"))
        self.player.convert()

        self.target = pygame.image.load(os.path.join("assets/sprites/target_old.png"))
        self.target.convert()

        pygame.font.init()
        self.myfont = pygame.font.SysFont("Comic Sans MS", 20)

        # Physics constants
        self.FPS = 60
        self.gravity = 0.08
        self.thruster_amplitude = 0.04
        self.diff_amplitude = 0.003
        self.thruster_mean = 0.04
        self.mass = 1
        self.arm = 25

        # Initialize variables
        (self.a, self.ad, self.add) = (0, 0, 0)
        (self.x, self.xd, self.xdd) = (400, 0, 0)
        (self.y, self.yd, self.ydd) = (400, 0, 0)
        self.xt = randrange(200, 600)
        self.yt = randrange(200, 600)

        # Initialize game variables
        self.target_counter = 0
        self.reward = 0
        self.time = 0
        self.time_limit = 20
        if self.mouse_target is True:
            self.time_limit = 1000

        # 2 action thrust amplitude and thrust difference in float values between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,))
        # 6 observations: angle_to_up, velocity, angle_velocity, distance_to_target, angle_to_target, angle_target_and_velocity
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,))

    def reset(self):
        # Reset variables
        (self.a, self.ad, self.add) = (0, 0, 0)
        (self.x, self.xd, self.xdd) = (400, 0, 0)
        (self.y, self.yd, self.ydd) = (400, 0, 0)
        self.xt = randrange(200, 600)
        self.yt = randrange(200, 600)

        self.target_counter = 0
        self.reward = 0
        self.time = 0

        return self.get_obs()

    def get_obs(self) -> np.ndarray:
        """
        Calculates the observations

        Returns:
            np.ndarray: The normalized observations:
            - angle_to_up : angle between the drone and the up vector (to observe gravity)
            - velocity : velocity of the drone
            - angle_velocity : angle of the velocity vector
            - distance_to_target : distance to the target
            - angle_to_target : angle between the drone and the target
            - angle_target_and_velocity : angle between the to_target vector and the velocity vector
        """
        angle_to_up = self.a / 180 * pi
        velocity = sqrt(self.xd**2 + self.yd**2)
        angle_velocity = self.ad
        distance_to_target = (
            sqrt((self.xt - self.x) ** 2 + (self.yt - self.y) ** 2) / 500
        )
        angle_to_target = np.arctan2(self.yt - self.y, self.xt - self.x)
        # Angle between the to_target vector and the velocity vector
        angle_target_and_velocity = np.arctan2(
            self.yt - self.y, self.xt - self.x
        ) - np.arctan2(self.yd, self.xd)
        distance_to_target = (
            sqrt((self.xt - self.x) ** 2 + (self.yt - self.y) ** 2) / 500
        )
        return np.array(
            [
                angle_to_up,
                velocity,
                angle_velocity,
                distance_to_target,
                angle_to_target,
                angle_target_and_velocity,
            ]
        ).astype(np.float32)

    def step(self, action):
        # Game loop
        self.reward = 0.0
        (action0, action1) = (action[0], action[1])

        # Act every 5 frames
        for _ in range(5):
            self.time += 1 / 60

            if self.mouse_target is True:
                self.xt, self.yt = pygame.mouse.get_pos()

            # Initialize accelerations
            self.xdd = 0
            self.ydd = self.gravity
            self.add = 0
            thruster_left = self.thruster_mean
            thruster_right = self.thruster_mean

            thruster_left += action0 * self.thruster_amplitude
            thruster_right += action0 * self.thruster_amplitude
            thruster_left += action1 * self.diff_amplitude
            thruster_right -= action1 * self.diff_amplitude

            # Calculating accelerations with Newton's laws of motions
            self.xdd += (
                -(thruster_left + thruster_right) * sin(self.a * pi / 180) / self.mass
            )
            self.ydd += (
                -(thruster_left + thruster_right) * cos(self.a * pi / 180) / self.mass
            )
            self.add += self.arm * (thruster_right - thruster_left) / self.mass

            self.xd += self.xdd
            self.yd += self.ydd
            self.ad += self.add
            self.x += self.xd
            self.y += self.yd
            self.a += self.ad

            dist = sqrt((self.x - self.xt) ** 2 + (self.y - self.yt) ** 2)

            # Reward per step survived
            self.reward += 1 / 60
            # Penalty according to the distance to target
            self.reward -= dist / (100 * 60)

            if dist < 50:
                # Reward if close to target
                self.xt = randrange(200, 600)
                self.yt = randrange(200, 600)
                self.reward += 100
                
                
                #
                self.target_counter+=1

            # If out of time
            if self.time > self.time_limit:
                done = True
                break

            # If too far from target (crash)
            elif dist > 1000:
                self.reward -= 1000
                done = True
                break

            else:
                done = False

            if self.render_every_frame is True:
                self.render("yes")

        info = {}

        return (
            self.get_obs(),
            self.reward,
            done,
            info,
        )

    def render(self, mode):
        # Pygame rendering
        pygame.event.get()
        self.screen.fill(0)
        self.screen.blit(
            self.target,
            (
                self.xt - int(self.target.get_width() / 2),
                self.yt - int(self.target.get_height() / 2),
            ),
        )
        player_copy = pygame.transform.rotate(self.player, self.a)
        self.screen.blit(
            player_copy,
            (
                self.x - int(player_copy.get_width() / 2),
                self.y - int(player_copy.get_height() / 2),
            ),
        )

        textsurface = self.myfont.render(
            "Collected: " + str(self.target_counter), False, (255, 255, 255)
        )
        self.screen.blit(textsurface, (20, 20))
        textsurface3 = self.myfont.render(
            "Time: " + str(int(self.time)), False, (255, 255, 255)
        )
        self.screen.blit(textsurface3, (20, 50))

        pygame.display.update()
        self.FramePerSec.tick(self.FPS)

    def close(self):
        pass


## TEST ENVIRONMENT WORKING

In [3]:
env = droneEnv(True, False)

obs = env.reset()
env.render("yes")

print("Observation space:")
print(env.observation_space)
print("")
print("Action space:")
print(env.action_space)
print("")
print("Action space sample:")
print(env.action_space.sample())

# Choose an action to execute n_steps times
action = np.array([0.0, 1.0])
n_steps = 10
for step in range(n_steps):
    print("Step {}".format(step + 1))
    obs, reward, done, info = env.step(action)
    print("obs=", obs, "reward=", reward, "done=", done)
    env.render("yes")
    if done:
        print("Done!", "reward=", reward)
        break


Observation space:
Box([-inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf], (6,), float32)

Action space:
Box([-1. -1.], [1. 1.], (2,), float32)

Action space sample:
[0.69754815 0.47666866]
Step 1
obs= [-0.03926991  0.00418868 -0.75        0.5318017   0.8013739   0.79181826] reward= -0.13825362982355913 done= False
Step 2
obs= [-0.14398967  0.03454228 -1.5         0.5316636   0.80162644  0.7626214 ] reward= -0.13822441168881405 done= False
Step 3
obs= [-0.31415927  0.1170206  -2.25        0.53107655  0.8026165   0.7145571 ] reward= -0.13806922357633705 done= False
Step 4
obs= [-0.5497787   0.27655566 -3.          0.52946705  0.80502886  0.6484174 ] reward= -0.13759285149245498 done= False
Step 5
obs= [-0.850848    0.53502405 -3.75        0.5259651   0.8095198   0.56514055] reward= -0.13648331002542924 done= False
Step 6
obs= [-1.2173672   0.907548   -4.5         0.5194007   0.8164096   0.46564546] reward= -0.13430199893417313 done= False
Step 7
obs= [-1.6493361   1.3969848  -5.

## TRAIN PPO AGENT

In [8]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
import wandb
from wandb.integration.sb3 import WandbCallback


run = wandb.init(
    project="CSP",
    sync_tensorboard=True,
    monitor_gym=True,
)

# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = droneEnv(False, False)
env = Monitor(env, log_dir)

# Create TD agent
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)



training_steps = 1_000_000

# Create checkpoint callback
checkpoint_callback = CheckpointCallback(
    save_freq=training_steps, save_path=log_dir, name_prefix="ppo_model_v6"
)

# Train the agent
model.learn(
    total_timesteps=training_steps,
    callback=[
        checkpoint_callback,
        WandbCallback(
            gradient_save_freq=100,
            model_save_path=f"models/{run.id}",
            model_save_freq=training_steps,
            verbose=2,
        ),
    ],
)

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Using cpu device
Wrapping the env in a DummyVecEnv.
Logging to tmp/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 41.5     |
|    ep_rew_mean     | -991     |
| time/              |          |
|    fps             | 712      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 40.9         |
|    ep_rew_mean          | -985         |
| time/                   |              |
|    fps                  | 355          |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0040013473 |
|    clip_fraction        | 0.0205       |
|    clip_range           | 0.2          |
|    entropy_loss         | 

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 43.9         |
|    ep_rew_mean          | -988         |
| time/                   |              |
|    fps                  | 358          |
|    iterations           | 11           |
|    time_elapsed         | 62           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0044342554 |
|    clip_fraction        | 0.0216       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.94        |
|    explained_variance   | 0.000243     |
|    learning_rate        | 0.0003       |
|    loss                 | 8.07e+04     |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.00326     |
|    std                  | 1.05         |
|    value_loss           | 1.69e+05     |
------------------------------------------
------------------------------------------
| rollout/ 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 42.7        |
|    ep_rew_mean          | -985        |
| time/                   |             |
|    fps                  | 362         |
|    iterations           | 20          |
|    time_elapsed         | 112         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.005374747 |
|    clip_fraction        | 0.0362      |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.05       |
|    explained_variance   | 3.53e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.6e+04     |
|    n_updates            | 190         |
|    policy_gradient_loss | -0.00339    |
|    std                  | 1.12        |
|    value_loss           | 1.44e+05    |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 44.4        |
|    ep_rew_mean          | -994        |
| time/                   |             |
|    fps                  | 367         |
|    iterations           | 29          |
|    time_elapsed         | 161         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.004978488 |
|    clip_fraction        | 0.0277      |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.12       |
|    explained_variance   | 1.15e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.06e+04    |
|    n_updates            | 280         |
|    policy_gradient_loss | -0.00352    |
|    std                  | 1.16        |
|    value_loss           | 1.21e+05    |
-----------------------------------------
-----------------------------------------
| rollout/                |       

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 240         |
|    ep_rew_mean          | 932         |
| time/                   |             |
|    fps                  | 349         |
|    iterations           | 461         |
|    time_elapsed         | 2698        |
|    total_timesteps      | 944128      |
| train/                  |             |
|    approx_kl            | 0.012015989 |
|    clip_fraction        | 0.128       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.815      |
|    explained_variance   | 0.194       |
|    learning_rate        | 0.0003      |
|    loss                 | 974         |
|    n_updates            | 4600        |
|    policy_gradient_loss | 0.000931    |
|    std                  | 0.371       |
|    value_loss           | 3.32e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |       

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 238          |
|    ep_rew_mean          | 937          |
| time/                   |              |
|    fps                  | 350          |
|    iterations           | 470          |
|    time_elapsed         | 2747         |
|    total_timesteps      | 962560       |
| train/                  |              |
|    approx_kl            | 0.0142571125 |
|    clip_fraction        | 0.164        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.795       |
|    explained_variance   | 0.275        |
|    learning_rate        | 0.0003       |
|    loss                 | 993          |
|    n_updates            | 4690         |
|    policy_gradient_loss | -0.000482    |
|    std                  | 0.366        |
|    value_loss           | 3.38e+03     |
------------------------------------------
-----------------------------------------
| rollout/  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 239         |
|    ep_rew_mean          | 953         |
| time/                   |             |
|    fps                  | 351         |
|    iterations           | 479         |
|    time_elapsed         | 2791        |
|    total_timesteps      | 980992      |
| train/                  |             |
|    approx_kl            | 0.012817298 |
|    clip_fraction        | 0.154       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.773      |
|    explained_variance   | 0.296       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.75e+03    |
|    n_updates            | 4780        |
|    policy_gradient_loss | -0.00518    |
|    std                  | 0.363       |
|    value_loss           | 3.27e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 239         |
|    ep_rew_mean          | 915         |
| time/                   |             |
|    fps                  | 352         |
|    iterations           | 488         |
|    time_elapsed         | 2833        |
|    total_timesteps      | 999424      |
| train/                  |             |
|    approx_kl            | 0.012937615 |
|    clip_fraction        | 0.12        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.781      |
|    explained_variance   | 0.359       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.3e+03     |
|    n_updates            | 4870        |
|    policy_gradient_loss | -0.0022     |
|    std                  | 0.367       |
|    value_loss           | 3.36e+03    |
-----------------------------------------


OSError: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\AVISH\\models\\5kda4cwh\\model.zip' -> 'C:\\Users\\AVISH\\wandb\\run-20231209_172614-5kda4cwh\\files\\model.zip'

## TEST TRAINED AGENT IN THE ENVIRONMENT

In [10]:
import torch as th
from stable_baselines3 import PPO
MODEL_PATH =r"C:\Users\AVISH\tmp\ppo_model_v1_1000000_steps.zip"

# Create and wrap the environment
env = droneEnv(True, False)

# Load the trained agent
model = PPO.load(MODEL_PATH, env=env)

# Evaluate the agent
for i in range(5):
    obs = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
    print("Episode reward", episode_reward)
    env.render("yes")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Episode reward 982.158490281774
Episode reward 1087.0984967706584
Episode reward 882.398535159258
Episode reward 781.6807484924246
Episode reward 777.6349398393277
